Which IDs are needed? </br>
- Get current date
- look up if file of last month that was downloaded
. </br>
. </br>
. </br>


In [1]:
import requests
from bs4 import BeautifulSoup as bs
import time
import os
import zipfile
import numpy as np
import pandas as pd
import glob
from datetime import datetime



In [2]:

MONTH_START = "2020-01" # Start month in the format yyyy-mm
MONTH_COUNT = 1 # sensor data will be downloaded for this amount of months
ROOT_URL = "https://archive.sensor.community/"
ROOT_DIR = '../data/SensorCommunity/'
WAIT_BETWEEN_DOWNLOADS = (0, 1)
SENSORS = [
    # 'bme280', 
    # 'bmp180', 
    # 'bmp280', 
    # 'dht22',
    # 'ds18b20', 
    # 'hpm', 
    # 'htu21d', 
    # 'pms1003', 
    # 'pms3003', 
    # 'pms5003', 
    # 'pms6003', 
    # 'pms7003', 
    # 'ppd42ns', 
    'sds011',
]
LAT_RANGE = [
    (53.013, 53.1456), 
    (50.030681, 50.205692),
]
# Bremen: (53.013, 53.1456)
# Frankfurt a. M.: (50.030681, 50.205692)
LON_RANGE = [
    (8.67, 8.9334), 
    (8.430634, 8.919868),
]

In [3]:
all_files = glob.glob(ROOT_DIR + "/*.csv")
today = datetime.today()

for sensor in SENSORS:
    sensor_files = []   
    for file in all_files:
        if sensor in file: # and not 'daily' in file:
            sensor_files.append(file)
    sensor_files.sort(reverse=True)

    start_date = None
    if 'daily' in sensor_files[0]:
        df_daily_last = pd.read_csv(sensor_files[0])
        start_date = pd.to_datetime(df_daily_last['timestamp']).dt.date.max()
        month = start_date.month
        start_date += pd.Timedelta(1, unit='days')            
    else:
        start_year = int(sensor_files[0].split('-')[0][-4:])
        start_month = int(sensor_files[0].split('_')[0][-2:]) + 1
        if start_month > 12:
            start_month = 1
            start_year += 1
        start_date = pd.to_datetime(f"{start_year}-{start_month}-01").date()
    dates = pd.date_range(start_date, today, freq='d')
    
    sensor_ids = pd.Series(dtype=int)
    for i in range(12):
        df_old = pd.read_csv(sensor_files[i])
        sensor_ids = pd.concat([sensor_ids, pd.Series(df_old['sensor_id'].unique())])
    sensor_ids = sensor_ids.unique()

    possible_urls = {}
    for date in dates:
        for sensor_id in sensor_ids:
            file_name = f"{date.strftime('%Y-%m-%d')}_{sensor}_sensor_{sensor_id}.csv"
            possible_urls[file_name] = ROOT_URL + str(date.strftime('%Y-%m-%d')) + '/' + file_name


    if not os.path.exists(os.path.join(ROOT_DIR, "daily")):
        os.mkdir(os.path.join(ROOT_DIR, "daily"))

    for name, url in possible_urls.items():
        csv_path = os.path.join(ROOT_DIR, "daily", name)
        # download .zip file if it doesn't exist yet
        if not os.path.isfile(csv_path):
            response = requests.get(url, timeout=50)
            if 'The requested URL was not found on this server.' not in str(response.content):
                with open(csv_path, 'wb') as f:
                    f.write(response.content)
                print(f"{name} downloaded")

    # TODO: if the last file is a 'daily' file, check if there are missing values
    #            -> if so: append corresponding values
    #            -> if not: make new file and append corresponding values
    #       if it's not a 'daily' file:
    #            -> make new file and append corresponding values
    #       check if there are further data that were not appended
    #            -> if so: make new file and append

    if 'daily' in sensor_files[0]:
        pass
    

2022-03-17_sds011_sensor_43744.csv downloaded
2022-03-17_sds011_sensor_20880.csv downloaded
2022-03-17_sds011_sensor_13862.csv downloaded
2022-03-17_sds011_sensor_67162.csv downloaded
2022-03-17_sds011_sensor_41149.csv downloaded
2022-03-17_sds011_sensor_19814.csv downloaded
2022-03-17_sds011_sensor_29548.csv downloaded
2022-03-17_sds011_sensor_23814.csv downloaded
2022-03-17_sds011_sensor_13291.csv downloaded
2022-03-17_sds011_sensor_31454.csv downloaded
2022-03-17_sds011_sensor_24677.csv downloaded
2022-03-17_sds011_sensor_34144.csv downloaded
2022-03-17_sds011_sensor_32333.csv downloaded
2022-03-17_sds011_sensor_30687.csv downloaded
2022-03-17_sds011_sensor_6757.csv downloaded
2022-03-17_sds011_sensor_31204.csv downloaded
2022-03-17_sds011_sensor_32186.csv downloaded
2022-03-17_sds011_sensor_30551.csv downloaded
2022-03-17_sds011_sensor_18421.csv downloaded
2022-03-17_sds011_sensor_64887.csv downloaded
2022-03-17_sds011_sensor_37405.csv downloaded
2022-03-17_sds011_sensor_33094.csv 

In [ ]:
all_files = glob.glob(ROOT_DIR + "/*.csv")
today = datetime.today()

for sensor in SENSORS:
    sensor_files = []   
    for file in all_files:
        if sensor in file: # and not 'daily' in file:
            sensor_files.append(file)
    sensor_files.sort(reverse=True)

    start_date = None
    if 'daily' in sensor_files[0]:
        df_daily_last = pd.read_csv(sensor_files[0])
        start_date = pd.to_datetime(df_daily_last['timestamp']).dt.date.max()
        month = start_date.month
        start_date += pd.Timedelta(1, unit='days')  

In [ ]:
# for sensor in SENSORS:
#     all_daily_files = glob.glob(os.path.join(ROOT_DIR, "daily") + "/*.csv")
#     all_daily_files
#     for file in all_files:
#         if sensor in file and 'daily' in file:
#             df_month = pd.read_csv(file)
#             for daily_file in all_daily_files:
#                 if sensor in daily_file:
#                     df_daily = pd.read_csv(daily_file)
#                     df_month = pd.concat([df_month, df_daily])
#             df_month.to_csv(file)


In [4]:
concat_csvs = pd.Series(dtype=object)
all_daily_files = glob.glob(os.path.join(ROOT_DIR, "daily") + "/*.csv")

for daily_file in all_daily_files:
    month = daily_file.split('/')[-1].split('_')[0][:-3]
    sens = daily_file.split('/')[-1].split('_')[1]
    concat_csvs = pd.concat([concat_csvs, pd.Series([month + '_' + sens])])
concat_csvs = concat_csvs.unique()
print(concat_csvs)

# for concat_csv in concat_csvs:
#     if not os.path.isfile(concat_csv + '.csv'):
#         if not os.path.isfile(concat_csv + '_daily.csv'):
#             df = pd.read_csv(all_daily_files[0], sep=';')
#             month = daily_file.split('/')[-1].split('_')[0][:-3]
#             sens = daily_file.split('/')[-1].split('_')[1]
#             df.to_csv(ROOT_DIR + month + '_' + sens + '_daily.csv')

# all_files = glob.glob(ROOT_DIR + "/*.csv")
# for file in all_files:
#     if 'daily' in file:
#         df_daily = pd.read_csv(file, index_col=0)
#         df_daily['timestamp'] = pd.to_datetime(df_daily['timestamp'])#.dt.date
#         max_date = df_daily['timestamp'].dt.date.max()
#         print(max_date)

# df_daily
# for daily_file in all_daily_files:
#     pass

['2022-03_sds011']


In [ ]:
sensor = 'sds011'
all_files = glob.glob(ROOT_DIR + "/*.csv")

sensor_files = []
for file in all_files:
    if sensor in file and not 'daily' in file:
        sensor_files.append(file)
sensor_files.sort(reverse=True)

last_year = int(sensor_files[0].split('-')[0][-4:])
last_month = int(sensor_files[0].split('_')[0][-2:])

next_year = last_year
next_month = last_month + 1
if next_month > 12:
    next_month = 1
    next_year += 1


In [ ]:
if next_month < 10:
    start_date = f"{next_year}-0{next_month}-01"
else:
    start_date = f"{next_year}-{next_month}-01"
start_date = datetime.strptime(start_date,"%Y-%m-%d")
today = datetime.today()
dates = pd.date_range(start_date, today, freq='d')
dates

In [ ]:
sensor_ids = pd.Series(dtype=int)
for i in range(12):
    df_old = pd.read_csv(sensor_files[i])
    sensor_ids = pd.concat([sensor_ids, pd.Series(df_old['sensor_id'].unique())])

sensor_ids = sensor_ids.unique()
sensor_ids

In [ ]:
# # find download links according to the sensors list and save them with file names
# url_dates = []
# for date in dates:
#     url_dates.append(ROOT_URL + f"{date.strftime('%Y-%m-%d')}/")

# url_dates

# # urls = []
# # names = []
# # for url_curr in url_dates:
# #     for sensor_id in sensor_ids:
# #         r = requests.get(url_curr)
# #         soup = bs(r.text)

# #         for i, link in enumerate(soup.findAll('a')):
# #             if '.csv' in str(link) and sensor in str(link) and str(sensor_id) in str(link):
# #                 url_download = url_curr + link.get('href')
# #                 urls.append(url_download)
# #                 names.append(soup.select('a')[i].attrs['href'])
# # print("Files to download:")
# # for file_name in names:
# #     print(file_name)
# # names_urls = zip(names, urls)

# # # # 2022-03-20_sds011_sensor_53243.csv
# # # for sensor_id in sensor_ids:
# # #     for date in dates:
# # #         file_name = f"{date.strftime('%Y-%m-%d')}_{sensor}_sensor_{sensor_id}.csv"
# # #         print(file_name)


In [ ]:
# # 2022-03-20_sds011_sensor_53243.csv
# file_names = []
# for sensor_id in sensor_ids:
#     for date in dates:
#         file_name = f"{date.strftime('%Y-%m-%d')}_{sensor}_sensor_{sensor_id}.csv"
#         file_names.append(file_name)

# print("got file_names")
    
# urls = []
# names = []
# for date in dates:
#     possible_urls = []
#     for sensor_id in sensor_ids:
#         possible_urls.append(
#             ROOT_URL
#             + str(date.strftime('%Y-%m-%d'))
#             + f"/{date.strftime('%Y-%m-%d')}_{sensor}_sensor_{sensor_id}.csv"
#         )

#     url_curr = ROOT_URL + str(date.strftime('%Y-%m-%d'))
#     r = requests.get(url_curr)
#     soup = bs(r.text)

#     for i, link in enumerate(soup.findAll('a')):
#         if link in possible_urls:
#             url_download = url_curr + link.get('href')
#             urls.append(url_download)
#             names.append(soup.select('a')[i].attrs['href'])
# print("Files to download:")
# for file_name in names:
#     print(file_name)
# names_urls = zip(names, urls)
# print(dict(names_urls))


In [ ]:
possible_urls = {}
for date in dates:
    for sensor_id in sensor_ids:
        file_name = f"{date.strftime('%Y-%m-%d')}_{sensor}_sensor_{sensor_id}.csv"
        possible_urls[file_name] = ROOT_URL + str(date.strftime('%Y-%m-%d')) + '/' + file_name
possible_urls

In [ ]:
if not os.path.exists(os.path.join(ROOT_DIR, "daily")):
    os.mkdir(os.path.join(ROOT_DIR, "daily"))

for name, url in possible_urls.items():
    csv_path = os.path.join(ROOT_DIR, "daily", name)
    # download .zip file if it doesn't exist yet
    if not os.path.isfile(csv_path):
        response = requests.get(url, timeout=50)
        if 'The requested URL was not found on this server.' not in str(response.content):
            with open(csv_path, 'wb') as f:
                f.write(response.content)
                print(f"{name} downloaded")
